In [1]:
from gedcom.element.individual import IndividualElement
from gedcom.parser import Parser
import gedcom.tags
import numpy as np
import pandas as pd
import math

from ipynb.fs.full.explorertheoph import *

gedcom_parser = Parser()

Information about Elizabeth II Alexandra Mary Windsor:


In [2]:
def shortest_path(graph, v1, v2):
    """
    like shortest_path1, but more efficient
    as it maintains the border incrementally

    graph : dictionnary which values are dictionnaries
    v1, v2 : FamilyElement, keys of graph
    """

    # keep track of what has been visited
    # with what distance, and from what vertex
    visited = {v1: (0, None)}
    # the edges at the border between
    # the visited and unvisited parts
    border_edges = set()
    # the vertex that was just selected
    selected_vertex = v1

    while True:
        # add to the border the edges that
        # go out of the last selected vertex
        # to unvisited
        # print(f"{selected_vertex=}")
        adj = graph.get(selected_vertex, {})
        for (dest, weight) in adj.items():
            if dest not in visited:
                border_edges.add((selected_vertex, dest))
        # remove from the border any edge that would
        # end at the newly_elected vertex
        border_edges = {
            (s, d) for (s, d) in border_edges
            if d != selected_vertex
        }
        # print(f"{border_edges=}")

        # out of luck, no path can be found
        # and border_edges is empty
        if not border_edges:
            print("no edges")
            return None

        # find the best tuple (edge, distance)
        shortest_length = math.inf
        shortest_edge = None
        for (s, d) in border_edges:
            w = graph[s][d]
            past_distance, _ = visited[s]
            dist = past_distance + w
            if dist <= shortest_length:
                shortest_length = dist
                shortest_edge = (s, d)

        # mark newly selected vertex
        best_src, best_dst = shortest_edge
        visited[best_dst] = (shortest_length, best_src)
        selected_vertex = best_dst

        # are we done ?
        if best_dst == v2:
            path = [v2]
            previous = best_src
            while previous:
                # print(f"inserting {previous}")
                path.insert(0, previous)
                previous = visited[previous][1]
            return shortest_length, path

#### Remark : form of graph
Graph gives close family members for each individual, i.e. : parents, siblings, grandparents, grandchildren, nephews, uncles and aunts. Those are family members separated by maximum one other individual. We must ensure that the direct distance between an individual and their uncle (for exemple) is inferior to the sum of distances between an individual and their father (for ex) and between the father and his brother. This way, the algorithm can decide which path is shorter.

The values of graph are tuples (weight, family link) where family links can be 'parent', 'sibling', 'grandparent', 'grandchild', 'nephew', uncle/aunt'. 

In [3]:
# not yet tested
# because needs graph in the proper form

def shortest_path_up(graph, v1, v2) :
    """
    Updated version of shortest_path
    Includes family relationship types

    graph : 
        dictionnary which values are dictionnaries
        neighbour vertexes are family members
        separated by maximum one (or two ?) vertex
        each item is a tuple (weight, 'family relationship')
    v1, v2 : 
        IndividualElement, keys of graph

    processus the path in order to give 
    the precise link between v1 and v2
    """

    # keep track of what has been visited
    # with what distance, and from what vertex
    visited = {v1: (0, None)}
    # the edges at the border between
    # the visited and unvisited parts
    border_edges = set()
    # the vertex that was just selected
    selected_vertex = v1

    while True:
        # add to the border the edges that
        # go out of the last selected vertex
        # to unvisited
        # print(f"{selected_vertex=}")
        adj = graph.get(selected_vertex, {})
        for (dest, value) in adj.items():
            if dest not in visited:
                border_edges.add((selected_vertex, dest))
        # remove from the border any edge that would
        # end at the newly_elected vertex
        border_edges = {
            (s, d) for (s, d) in border_edges
            if d != selected_vertex
        }
        # print(f"{border_edges=}")

        # out of luck, no path can be found
        # and border_edges is empty
        if not border_edges:
            print("no edges")
            return None

        # find the best tuple (edge, distance)
        shortest_length = math.inf
        shortest_edge = None
        for (s, d) in border_edges:
            w = graph[s][d][0]
            past_distance, _ = visited[s]
            dist = past_distance + w
            if dist <= shortest_length:
                shortest_length = dist
                shortest_edge = (s, d)

        # mark newly selected vertex
        best_src, best_dst = shortest_edge
        visited[best_dst] = (shortest_length, best_src)
        selected_vertex = best_dst

        # are we done ?
        if best_dst == v2:
            # path contains the vertex and family link
            first = [best_dst, graph[best_src][best_dst][1]]
            path = [first]
            previous = best_src
            surprevious = visited[previous][1]
            while previous:
                # print(f"inserting {previous}")
                next = [previous, graph[surprevious][previous][1]]
                path.insert(0, next)
                previous = surprevious
                surprevious = visited[previous][1]
            return shortest_length, path

### Interpretation of path
We must end up with a formulation like : "v2 is the father of the grandmother of v1." This implies finding genders of all individuals in the path, and maybe doing some simplifications such as father of father = grandfather. 

##### Question
Should we rather say "niece" or "daugther of brother" ? "Niece" is more easy to read, but "daughter of brother" is 100% precise. Ask Pierre-Antoine

In [4]:
# names of family links
links = {'parents' : ('father', 'mother'), 
         'sibling' : ('brother', 'sister'), 
         'grandparent' : ('grandfather', 'grandmother'),
         'grandchild' : ('grandson', 'granddaughter'), 
         'nephew' : ('nephew', 'niece'), 
         'uncle/aunt' : ('uncle', 'aunt')
        }

In [5]:
def gendered_link(tag, link) :
    """
    Adds gender to family link

    Parameters
    ---
    tag : str
        tag of IndividualElement
    link : str
        family link
    Returns 
    ---
    str 
        link with gender
    """

    indvidual = find_Element(tag) 
    
    if individual.get_gender() == "M" :
        return links[links][0]
    else :
        return links[links][1]

In [6]:
def print_path(path) :
    """
    Prints interprated path

    Parameters
    ---
    path : list of tuples
        path between two individuals
        tuple : (individual tag, family link)

    Returns
    ---
    int 

    """
    ind, link = path[0]
    print (v2 + "is the" + gendered_link(path[0][0], path[0][1]))
    for i in range (1, len(path)) :
        ind, link = path[i]
        print("of the" + gendered_link(ind, link))
    print ("of" + v1)

    # les print s'afficheront sûrement chacun à la ligne
    # il faut trouver une commande qui empêche ça


## Creation of graph

In [7]:
def get_IndivFamily_DataFrame(file_path='Queen_Eliz_II.ged'):
    """
    Creates DataFrame of children & spouse families keys of all individuals 
    indexed by their keys, from a gedcom file.

    Parameters
    ---
    file_path : str
        path of the gedcom file

    Returns 
    ---
    pd.DataFrame 
        dataframe of children & spouse families keys of individuals
    """
    gedcom_parser.parse_file(file_path)
    root_child_elements = gedcom_parser.get_root_child_elements()
    
    T = []
    
    #Go through indivduals and get their families
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            L = [element.get_pointer()]
            for child_element in element.get_child_elements() :
                if child_element.get_tag() == gedcom.tags.GEDCOM_TAG_FAMILY_SPOUSE :
                    L += [child_element.get_value()]
                elif child_element.get_tag() == gedcom.tags.GEDCOM_TAG_FAMILY_CHILD :
                    L += [child_element.get_value()]
            T += [L]

    #Add NaN where information is missing
    full_T = [line+['NaN']*(3-len(line)) for line in T]

    #Create the DataFrame
    df = pd.DataFrame(
    {
        'INDI' : [full_T[k][0] for k in range(len(full_T))],
        'FAMS' : [full_T[k][1] for k in range(len(full_T))],
        'FAMC' : [full_T[k][2] for k in range(len(full_T))],
    })

    return df

In [8]:
# Tests

print(get_IndivFamily_DataFrame().head())

df = get_IndivFamily_DataFrame()
df1 = df.set_index('INDI',inplace=False)
print(df1.at['@I11262@','FAMC'])

     INDI    FAMS    FAMC
0  @I101@  @F285@  @F286@
1  @I103@  @F286@  @F287@
2  @I155@   @F78@     NaN
3  @I168@   @F75@   @F76@
4  @I169@   @F75@  @F209@
NaN


### Family graph

In [9]:
def get_FamLinks_DataFrame(file_path='Queen_Eliz_II.ged'):

    df = get_IndivFamily_DataFrame(file_path)

    df1 = df.set_index('INDI',inplace=False)

    df2 = pd.DataFrame(
    {
        'FAM' : df['FAMS'].drop_duplicates(),
    })
    df2['1FAMS'] = np.NaN
    df2.set_index('FAM',inplace=True)
    N_max = 1

    for family in df2.index :
        N = 0

        for indi in df[df['FAMS'] == f'{family}']['INDI'] :
            N += 1
            if N > N_max :
                N_max = N
                df2[f'{N_max}FAMS'] = np.NaN
            df2.at[f'{family}',f'{N}FAMS'] = f'{indi}'
            #df2.at[f'{family}',f'{N}FAMS'] = df1.at[f'{indi}','FAMC']
        
        for indi in df[df['FAMC'] == f'{family}']['INDI'] :
            N += 1
            if N > N_max :
                N_max = N
                df2[f'{N_max}FAMC'] = np.NaN
            df2.at[f'{family}',f'{N}FAMC'] = f'{indi}'
            #df2.at[f'{family}',f'{N}FAMC'] = df1.at[f'{indi}','FAMS']
    
    return df2

In [10]:
def build_FamGraph(file_path='Queen_Eliz_II.ged'):

    g = {}
    df = get_FamLinks_DataFrame(file_path)

    for FAM1 in df.index :
        g[FAM1] = {}
        for FAM2 in df.loc[FAM1] :
            if f'{FAM2}' != 'nan' :
                g[FAM1][f'{FAM2}'] = 1
    return g

In [11]:
# Test
shortest_path(build_FamGraph(), '@F76@', '@F6017@')

no edges


### Individual graph

In [43]:
def get_IndivLinks_DataFrame(file_path='Queen_Eliz_II.ged'):

    df = get_IndivFamily_DataFrame(file_path)

    df1 = df.set_index('INDI',inplace=False)

    df2 = pd.DataFrame(
    {
        'INDI' : df1.index,
    })
    df2['PARENT1'] = np.NaN
    df2.set_index('INDI',inplace=True)

    for indiv1 in df2.index :

        #Parents : FAMC of the character = FAMS of parents
        N_parents = 0

        for indiv2 in df[df['FAMS'] == df1.at[f'{indiv1}','FAMC']]['INDI'] :
            N_parents += 1
            df2.at[f'{indiv1}',f'PARENT{N_parents}'] = f'{indiv2}'


        #Childrens : FAMS of the character = FAMC of childrens
        N_childrens = 0

        for indiv3 in df[df['FAMC'] == df1.at[f'{indiv1}','FAMS']]['INDI'] :
            N_childrens += 1
            df2.at[f'{indiv1}',f'CHILD{N_childrens}'] = f'{indiv3}'

        #Siblings : FAMC of character = FAMC of siblings
        N_siblings = 0

        for indiv4 in df[df['FAMC'] == df1.at[f'{indiv1}','FAMC']]['INDI'] :
            if indiv4 != indiv1 and df1.at[f'{indiv1}','FAMC'] != 'NaN':
                N_siblings += 1
                df2.at[f'{indiv1}',f'SIBLING{N_siblings}'] = f'{indiv4}'


    for indiv5 in df2.index :
        
        #Grand parents : parents of the character's parents
        N_gp = 0

        for parent in df2.loc[f'{indiv5}',['PARENT1','PARENT2']] :
            if not pd.isna(parent):
                for gparent in df2.loc[f'{parent}',['PARENT1','PARENT2']] :
                    if not pd.isna(gparent):
                        N_gp += 1
                        df2.at[f'{indiv5}',f'GRAND_PARENT{N_gp}'] = f'{gparent}'
        
        #Grand child : childrens of the character's childrens
        N_gc = 0

        for child in df2.loc[f'{indiv5}',[f'CHILD{i}' for i in range(1, N_childrens + 1)]] :
            if not pd.isna(child):
                for gchild in df2.loc[f'{child}',[f'CHILD{j}' for j in range(1, N_childrens + 1)]] :
                    if not pd.isna(gchild):
                        N_gc += 1
                        df2.at[f'{indiv5}',f'GRAND_CHILD{N_gc}'] = f'{gchild}'
        
        #Nephews : childrens of the character's siblings
        N_nephews = 0

        for sibling in df2.loc[f'{indiv5}',[f'SIBLING{k}' for k in range(1, N_siblings + 1)]] :
            if not pd.isna(sibling):
                for child in df2.loc[f'{sibling}',[f'CHILD{l}' for l in range(1, N_childrens + 1)]] :
                    if not pd.isna(child):
                        N_nephews += 1
                        df2.at[f'{indiv5}',f'NEPHEW{N_nephews}'] = f'{child}'

    df2 = df2.reindex(sorted(df2.columns), axis=1)
    return N_siblings

In [44]:
pd.set_option('display.max_rows', 10)
get_IndivLinks_DataFrame()

0

In [ ]:
def build_IndGraph(file_path='Queen_Eliz_II.ged'):

    g = {}
    df = get_IndivLinks_DataFrame(file_path)

    for IND1 in df.index :
        g[IND1] = {}
        for IND2 in df.loc[IND1] :
            if f'{IND2}' != 'nan' :
                g[IND1][f'{IND2}'] = 1
    return g

In [ ]:
# Test : graph creation
graph = build_IndGraph()

In [ ]:
# Test (for one single value in graph)
v1 = '@I390@'
v2 = '@I336@'

shortest_path_up(graph,v1,v2)

TypeError: 'int' object is not subscriptable